In [1]:
import pandas as pd
import numpy as np

from industries import industry_titles

# #lists of variables to drop during timeline construction
# dropcols_str = {2001:['2000.25', '2000.5', '2000.75', '2001.0', '2001.25', '2001.5', '2001.75', '2002.0', '2002.25', '2002.5', '2002.75', '2003.0', '2003.25', '2003.5', '2003.75', '2004.0', '2004.25', '2004.5', '2004.75', '2005.0', '2005.25', '2005.5', '2005.75', '2006.0', '2006.25', '2006.5', '2006.75', '2007.0', '2007.25', '2007.5', '2007.75', '2008.0', 'nadir', 'nadir_qtr', 'new', 'pre_peak', 'post_peak','pre_peak_qtr', 'post_peak_qtr', 'recovery', 'recovery_list'],
# 2008:['2007.25', '2007.5', '2007.75', '2008.0', '2008.25', '2008.5', '2008.75', '2009.0', '2009.25', '2009.5', '2009.75', '2010.0', '2010.25', '2010.5', '2010.75', '2011.0', '2011.25', '2011.5', '2011.75', '2012.0', '2012.25', '2012.5', '2012.75', '2013.0', '2013.25', '2013.5', '2013.75', '2014.0', '2014.25', '2014.5', '2014.75', '2015.0', '2015.25', '2015.5', '2015.75', '2016.0', '2016.25', '2016.5', '2016.75', '2017.0', '2017.25', '2017.5', '2017.75', '2018.0', '2018.25', '2018.5', '2018.75', '2019.0', '2019.25', '2019.5', '2019.75', '2020.0', 'nadir', 'nadir_qtr', 'nadir_time', 'new', 'pre_peak', 'post_peak',  'pre_peak_qtr', 'pre_peak_time', 'post_peak_qtr', 'post_peak_time' 'recovery', 'recovery_list'],
# 'full':['2000.25', '2000.5', '2000.75', '2001.0', '2001.25', '2001.5', '2001.75', '2002.0', '2002.25', '2002.5', '2002.75', '2003.0', '2003.25', '2003.5', '2003.75', '2004.0', '2004.25', '2004.5', '2004.75', '2005.0', '2005.25', '2005.5', '2005.75', '2006.0', '2006.25', '2006.5', '2006.75', '2007.0', '2007.25', '2007.5', '2007.75', '2008.0', '2008.25', '2008.5', '2008.75', '2009.0', '2009.25', '2009.5', '2009.75', '2010.0', '2010.25', '2010.5', '2010.75', '2011.0', '2011.25', '2011.5', '2011.75', '2012.0', '2012.25', '2012.5', '2012.75', '2013.0', '2013.25', '2013.5', '2013.75', '2014.0', '2014.25', '2014.5', '2014.75', '2015.0', '2015.25', '2015.5', '2015.75', '2016.0', '2016.25', '2016.5', '2016.75', '2017.0', '2017.25', '2017.5', '2017.75', '2018.0', '2018.25', '2018.5', '2018.75', '2019.0', '2019.25', '2019.5', '2019.75', '2020.0', 'nadir', 'nadir_qtr', 'new', 'pre_peak', 'post_peak','pre_peak_qtr', 'post_peak_qtr', 'recovery', 'recovery_list']}
# dropcols_flt = {2001:[2000.25, 2000.5, 2000.75, 2001.0, 2001.25, 2001.5, 2001.75, 2002.0, 2002.25, 2002.5, 2002.75, 2003.0, 2003.25, 2003.5, 2003.75, 2004.0, 2004.25, 2004.5, 2004.75, 2005.0, 2005.25, 2005.5, 2005.75, 2006.0, 2006.25, 2006.5, 2006.75, 2007.0, 2007.25, 2007.5, 2007.75, 2008.0, 'nadir', 'nadir_qtr', 'new', 'pre_peak', 'post_peak', 'pre_peak_qtr', 'post_peak_qtr', 'recovery', 'recovery_list'],
# 2008:[2007.25, 2007.5, 2007.75, 2008.0, 2008.25, 2008.5, 2008.75, 2009.0, 2009.25, 2009.5, 2009.75, 2010.0, 2010.25, 2010.5, 2010.75, 2011.0, 2011.25, 2011.5, 2011.75, 2012.0, 2012.25, 2012.5, 2012.75, 2013.0, 2013.25, 2013.5, 2013.75, 2014.0, 2014.25, 2014.5, 2014.75, 2015.0, 2015.25, 2015.5, 2015.75, 2016.0, 2016.25, 2016.5, 2016.75, 2017.0, 2017.25, 2017.5, 2017.75, 2018.0, 2018.25, 2018.5, 2018.75, 2019.0, 2019.25, 2019.5, 2019.75, 2020.0, 'nadir', 'nadir_qtr', 'nadir_time', 'new', 'pre_peak', 'post_peak',  'pre_peak_qtr', 'pre_peak_time', 'post_peak_qtr', 'post_peak_time' 'recovery', 'recovery_list'],
# 'full':[2000.25, 2000.5, 2000.75, 2001.0, 2001.25, 2001.5, 2001.75, 2002.0, 2002.25, 2002.5, 2002.75, 2003.0, 2003.25, 2003.5, 2003.75, 2004.0, 2004.25, 2004.5, 2004.75, 2005.0, 2005.25, 2005.5, 2005.75, 2006.0, 2006.25, 2006.5, 2006.75, 2007.0, 2007.25, 2007.5, 2007.75, 2008.0, 2008.25, 2008.5, 2008.75, 2009.0, 2009.25, 2009.5, 2009.75, 2010.0, 2010.25, 2010.5, 2010.75, 2011.0, 2011.25, 2011.5, 2011.75, 2012.0, 2012.25, 2012.5, 2012.75, 2013.0, 2013.25, 2013.5, 2013.75, 2014.0, 2014.25, 2014.5, 2014.75, 2015.0, 2015.25, 2015.5, 2015.75, 2016.0, 2016.25, 2016.5, 2016.75, 2017.0, 2017.25, 2017.5, 2017.75, 2018.0, 2018.25, 2018.5, 2018.75, 2019.0, 2019.25, 2019.5, 2019.75, 2020.0, 'nadir', 'nadir_qtr', 'new', 'pre_peak', 'post_peak', 'pre_peak_qtr', 'post_peak_qtr', 'recovery', 'recovery_list']}

#schema for importing dataframe
schema_dict = { 'area_fips':str,  'own_code':str,  'industry_code':str,  'agglvl_code':str,  'size_code':str,  'year':int,  'qtr':int,  'disclosure_code':str, 'area_title':str,  'own_title':str,  'industry_title':str,  'agglvl_title':str,  'size_title':str,  'qtrly_estabs':int,  'month1_emplvl':int,  'month2_emplvl':int,  'month3_emplvl':int,  'total_qtrly_wages':int,  'taxable_qtrly_wages':int,  'qtrly_contributions':int,  'avg_wkly_wage':int,  'lq_disclosure_code':str,  'lq_qtrly_estabs':float,  'lq_month1_emplvl':float,  'lq_month1_emplv2':float,  'lq_month1_emplv3':float,  'lq_total_qtrly_wages':float,  'lq_taxable_qtrly_wages':float,  'lq_qrtly_contributions':float,  'oty_disclosure_code':str,  'oty_qtrly_estabs':int,  'oty_qtrly_estabs_pct_chg':float,  'oty_month1_emplvl_chg':int,  'oty_month1_emplvl_pct_chg':float,  'oty_month2_emplv_chg':int,  'oty_month2_emplvl_pct_chg':float,  'oty_month3_emplvl_chg':int,  'oty_month3_emplvl_pct_chg':float,  'oty_total_qtrly_wages_chg':int,  'oty_total_qtrly_wages_pct_chg':float,  'oty_taxable_qtrly_wages_chg':int,  'oty_taxable_qtrly_wages_pct_chg':float,  'oty_qrtly_contributions_chg':int,  'oty_qrtly_contributions_pct_chg':float,  'oty_avg_wkly_wage_chg':int,  'oty_avg_wkly_wage_pct_chg':float} 

#unused columns from QCEW
import_drop = ['own_code',  'size_code',  'disclosure_code',  'own_title',  'size_title',  'lq_disclosure_code', 'oty_disclosure_code',  'oty_month1_emplvl_chg',  'oty_month2_emplvl_chg',  'oty_month3_emplvl_chg',  'oty_total_qtrly_wages_chg',  'oty_taxable_qtrly_wages_chg',  'oty_qtrly_contributions_chg',  'oty_avg_wkly_wage_chg',  'lq_qtrly_estabs_count',  'lq_month1_emplvl',  'lq_month2_emplvl',  'lq_month3_emplvl',  'lq_total_qtrly_wages',  'lq_taxable_qtrly_wages',  'lq_qtrly_contributions',  'oty_qtrly_estabs_count_chg',  'oty_qtrly_estabs_count_pct_chg',  'oty_month1_emplvl_pct',  'oty_month2_emplvl_pct',  'oty_month3_emplvl_pct',  'oty_total_qtrly_wages_pct',  'oty_taxable_qtrly_wages_chg',  'oty_qtrly_contributions_pct',  'oty_avg_wkly_wage_pct',  'oty_taxable_qtrly_wages_chg.1',  'lq_avg_wkly_wage',  'taxable_qtrly_wages',  'qtrly_contributions']

merge_drop = ['nadir', 'nadir_qtr', 'new', 'pre_peak', 'post_peak','pre_peak_qtr', 'post_peak_qtr', 'recovery', 'recovery_list']


def filepath(variable = 'month3_emplvl', dimension = 'area', charttype = 'basic', recession = 2001, filetype = 'json'):
    '''
    creates the filepath to load/save dataframes

        Parameters: 
            variable (str): indicates if the loaded/saved file will contain employment, wage, or firm data. Must be one of ['month3_emplvl' (default), 'avg_wkly_wage', 'qtrly_estabs_count']. 
            dimension (str): indicates if the loaded/saved file will construct area or industry timelines. Must be one of ['area', 'timeline'].
            charttype (str): indicates what type of timeline will be loaded/saved. Differentiation required for differing chart types in charting.py. Must be one of ['basic', 'target', 'proprtional'].
            recession (str): indicates the timeframe in question. Must be one of [2001, 2008, 'full']. 
            filetype (str): determines what type of file will be saved/loaded. Must be one of ['json', 'csv']. Use 'csv' for loading raw QCEW files.

        Returns:
            filepath (str): path to load appropriate file
    '''
    filename = dim_abbr[dimension] + "_" + var_abbr[variable] + "_" + str(recession) + "." + filetype 
    filepath = "../data/timelines/" + charttype + "/" + dimension + "/" + var_abbr[variable] + '/' +  filename 
    return filepath

def add_qtrid(df):
    '''
    Adds a column for the year and quarter. Needed for indexing.

        Parameters: 
            df (pandas dataframe)

        Returns:
            df (pandas dataframe): dataframe with column added
    '''
    df['qtrid'] = df['year'] + (df['qtr']/4)
    return df

def import_one(year, dimension = 'area'):
    '''
    Constructs a dataframe from a single year's worth of data.
    
        Parameters: 
            year (str): year of data to be imported
            dimension (str): dimension of data to import. Must be 'area' or 'industry'. Default = 'area'
        
        Returns:
            df (pandas dataframe)
    '''
    filepath = '../data/' + dimension + '_files/' + str(year) + '.csv'
    #all relevant CSVs should be named with only the year
    df = pd.read_csv(filepath, dtype = schema_dict)
    #removes redundant entries in industry files
    if dimension == 'industry':
        df = df[df.own_code != 8]
        df = df[df.own_code != 9]
    for column in import_drop:
        if column in df.columns:
            df = df.drop([column], axis = 1)
    #removes industry columns from area data
    if dimension == 'area':
        df = df.drop(columns = ['industry_code', 'industry_title'])
    #removes area columns from industry data
    elif dimension == 'industry':
        df = df.drop(columns = ['area_fips', 'area_title'])
    return df

def import_all(years, dimension = 'area'):
    '''
    Combines years of data into a single dataframe, adds qtrid

        Parameters: 
            years (list)
            dimension (str): dimension of data to import. Must be 'area' or 'industry'. Default = 'area'

        Returns: 
            df (dataframe)
    '''
    df = import_one(years[0], dimension)
    for year in years[1:]:
        df = df.append(import_one(year, dimension))
    #replaces irregular industry codes and converts to integer
    if dimension == 'industry':
        df['industry_code'] = df['industry_code'].str.replace('31-33','31')
        df['industry_code'] = df['industry_code'].str.replace('44-45','44')
        df['industry_code'] = df['industry_code'].str.replace('48-49','48').astype('int32')
    #adds qtrid column
    df = add_qtrid(df)
    return df

def basic_timeline(variable = 'month3_emplvl', dimension = 'area', recession = 2001, save = False):
    '''
    Produces a dataframe of the indicated recession timeline.

        Parameters: 
            variable (str): determines what economic indicator will be used in the timeline. Must be one of ['month3_emplvl' (employment), 'avg_wkly_wage' (wages), 'qtrly_estabs_count'(firms)]
            dimension (str): dimension of data to import. Must be 'area' or 'industry'. Default = 'area'.
            recession (int or 'full'): recession timeline to compute.
            save (bool): determines if a json file will be generated and saves locally.
    
        Returns: 
            df (pandas dataframe)
            exported json file
    '''
    #creates dataframe of all years in recession
    df = import_all(recessions_int[recession], dimension)

    #set indicies, drop unhelpful rows
    if dimension == 'area':
        df = df[~df['area_fips'].str.contains("999")]
        index = ['area_fips', 'area_title']
    elif dimension == 'industry':
        index = ['industry_code', 'industry_title']
        #correct for changes in NAICS classification
        df['industry_title'] = df['industry_code'].apply(lambda x: industry_titles[x])
    
    
    #pivots the table to arrange quarters in columns, drops extraneous variables.
    df = df.pivot_table(columns = 'qtrid', values = variable, index = index, aggfunc = np.sum)
    df = df.reset_index()
    
        #fill nans
    df = df.fillna(0)
    
    #export the data
    if save:
        savepath = filepath(variable = variable, dimension = dimension, recession = recession, filetype = 'json')
            
            # "data/timelines/basic" + dim_abbr[dimension] + "_" + var_abbr[variable] + "_" + str(recession) + ".json"
        df.to_json(savepath)
    return df

def target_timeline(variable = 'month3_emplvl', dimension = 'area', recession = 2001, save = False, loadjson = False):
    '''
    Produces a dataframe of the indicated recession timeline with derived target variables.
    WARNING: Long processing time required.

    Parameters: 
        variable (str): determines what economic indicator will be used in the timeline. Must be one of ['month3_emplvl' (employment), 'avg_wkly_wage' (wages), 'qtrly_estabs_count'(firms)]
        dimension (str): dimension of data to import. Must be 'area' or 'industry'. Default = 'area'.
        recession (int): recession timeline to compute. Default = 2001. Will pass if 'full'.
        save (bool): determines if a json file will be generated and saves locally. Default = False
        loadjson (bool): determines if basic timeline will be loaded from previously saved file(True) or created anew (False). Existing basic json files will be overwritten if save = True. Default = False

    Returns: 
        df (pandas dataframe)
        exported json file
    '''

    if recession == 'full':
        pass

    recession = Recession(recession)
    
    if loadjson:
        loadpath = filepath(variable = variable, dimension = dimension, charttype= 'basic', recession = recession.event_year, filetype = 'json')
        # filepath =  "data/timelines/basic/" + dim_abbr[dimension] + "_" + var_abbr[variable] + "_" + str(recession) + ".json"
        df = pd.read_json(loadpath)
    else:
        df = basic_timeline(variable = variable, dimension = dimension, recession = recession.event_year, save = save)
    
    if dimension == 'area':
        df = df[~df['area_fips'].str.contains("999")]
        index = ['area_fips', 'area_title']
    elif dimension == 'industry':
        index = ['industry_code', 'industry_title']


    #creates a secondary dataframe with only timeline variables
    df2 = df.drop(columns = index)
    df2 = df2.reset_index()
    
    #drops the index so that all columns are free of any type mismatches
    df2 = df2.drop(columns = 'index')
    df2 = df2.fillna(0)

    #specifies the lowest numbers during the recession. Disregards quarters before the recession event. 
    df2['nadir'] = df2.iloc[:,6:].min(axis=1)

    #specifies which quarter the nadir occured.
    df2['nadir_time'] = (df2.iloc[:,6:].idxmin(axis=1).apply(lambda x: df.columns.get_loc(x)))-1
    
    df2['nadir_qtr'] =  df2['nadir_time'] / 4 + recession.years[0]
    
    #creates a column to store indices for lookup.
    df2['new'] = [df2.iloc[i].values for i in df.index]
    
    #specifies the highest numbers *before* the nadir.
    df2['pre_peak'] = df2.apply(lambda x: max(x['new'][0:x['nadir_time']]), axis=1)
    
    #specifies the highest numbers *after* the nadir
    df2['post_peak'] = df2.apply(lambda x: max(x['new'][x['nadir_time']:]), axis=1) 
    
    #specifies which quarter the pre-peak occurred.
    df2['pre_peak_time'] = pd.Series([s[i] for i, s in zip(df2.index, df2['pre_peak'].apply(
        lambda x: [i for i in (df2.iloc[:,0:-6] == x)
                .idxmax(axis=1)]))]).apply(lambda x: df2.columns.get_loc(x)) + 1
    
    df2['pre_peak_qtr'] = df2['pre_peak_time'] / 4 + recession.years[0]

    #specifies which quarter the post-peak occurred.
    df2['post_peak_time'] = pd.Series([s[i] for i, s in zip(df2.index, df2['post_peak'].apply(
        lambda x: [i for i in (df2.iloc[:,0:-6] == x)
                .idxmax(axis=1)]))]).apply(lambda x: df2.columns.get_loc(x)) + 1

    df2['post_peak_qtr'] = df2['post_peak_time'] / 4 + recession.years[0]

    #PRIMARY TARGET: did the area/industry achieve it's pre-recession peak?
    df2['recovery'] = (df2['post_peak'] >= df2['pre_peak']) *1
    
    #create another dataframe to calculate the # of quarters to recover- only contains recovered datapoints
    df3 = df2[df2['recovery'] == 1]
    
    #creates a column to derive the recovery quarter- list of boolean values
    df3['recovery_list'] = df3.apply(lambda x: (x['new'][x['nadir_time']:] >= x['pre_peak']), axis=1)

    #creates a column for the number or quarters until the results pass the pre-peak high, since the nadir
    df3['recovery_time'] = df3['recovery_list'].apply(lambda x: list(x).index(True)) + 1

    df3['recovery_qtr'] = (df3['nadir_time'] + df3['recovery_time']) / 4 + recession.years[0]

    #drops all redunant fields from the third dataframe
    dropcols = merge_drop
    if loadjson:
        for quarter in recession.quarters:
            dropcols.append(str(quarter))
    else:
        dropcols.extend(recession.quarters)

    #creates a new dataframe to store derived fields
    df_new = df2[['nadir', 'nadir_qtr', 'nadir_time', 'pre_peak', 'pre_peak_time', 'pre_peak_qtr', 'post_peak', 'post_peak_time', 'post_peak_qtr', 'recovery']]

    #adds the recovery quarter column from the third dataframe
    df_new = df_new.join(df3, how = 'left', rsuffix = '_recov')

    #puts the computed points in a dataframe, joins with timeline
    df = df.join(df_new, how = 'outer', rsuffix = '_derive')
    
    #How many quarters did the jobs numbers decline?
    df['decline_time'] = (df['nadir_time'] - df['pre_peak_time'])

    df['growth_time'] = (df['post_peak_time'] - df['nadir_time'])
    
    #Different in before/after jobs numbers
    df['delta'] = df['post_peak'] - df['pre_peak']

    #remove extra columns
    df.drop(columns = ['nadir_time_recov', 'pre_peak_time_recov', 'post_peak_time_recov'], inplace = True)
    
    #reorders columns for easier organization
    col_order = index
    col_order.extend(recession.quarters)
    col_order.extend(['pre_peak', 'nadir', 'post_peak', 'recovery', 'delta', 'pre_peak_time', 'decline_time', 'nadir_time', 'post_peak_time', 'recovery_time', 'growth_time', 'pre_peak_qtr', 'nadir_qtr', 'post_peak_qtr', 'recovery_qtr', ])
    df = df.reindex(columns = col_order)

    #export the data
    if save:
        savepath = filepath(variable = variable, dimension = dimension, charttype= 'target', recession = recession.event_year, filetype = 'json')
        df.to_json(savepath)
    return df

def proportional_timeline(variable = 'month3_emplvl', dimension = 'area', recession = 2001, save = False):
    '''
    Produces a dataframe of the indicated recession timeline as a percentage of the pre-peak. Useful in recreating "scariest chart".

    Does not allow for creating new target timelines (too intensive).

        Parameters: 
            variable (str): determines what economic indicator will be used in the timeline. Must be one of ['month3_emplvl' (employment), 'avg_wkly_wage' (wages), 'qtrly_estabs_count'(firms)]
            dimension (str): dimension of data to import. Must be 'area' or 'industry'. Default = 'area'.
            recession (int or 'full'): recession timeline to compute. Default = 2001. Will exit if recession = 'full'.
            save (bool): determines if a json file will be generated and saved locally. Default = False
    
        Returns: 
            df (pandas dataframe)
            exported json file
    '''

    if recession == 'full':
        pass
    recession = Recession(recession)
    # filepath =  "data/timelines/targets/" + dim_abbr[dimension] + "_" + var_abbr[variable] + "_" + str(recession) + ".json" 
    loadpath = filepath(variable = variable, dimension = dimension, charttype= 'target', recession = recession.event_year, filetype = 'json')
    df = pd.read_json(loadpath)
    df = df.drop(columns =  ['nadir', 'nadir_qtr', 'pre_peak_qtr', 'post_peak', 'post_peak_qtr', 'recovery', 'recovery_qtr', 'decline', 'delta'])
    count = -6
    drop_list = ['pre_peak']
    for column in df.columns [2:-1]:
        df[count] = (df[column] - df['pre_peak']) / df['pre_peak']
        count += 1
        drop_list.append(column)
    df.drop(columns = drop_list, inplace= True)
    if save:
        # savepath =  "data/timelines/proportional/" + dim_abbr[dimension] + "_" + var_abbr[variable] + "_" + str(recession) + ".json"
        savepath = filepath(variable = variable, dimension = dimension, charttype= 'proportional', recession = recession.event_year, filetype = 'json')
        df.to_json(savepath) 
    return df

def export_all(basic = False, target = False, proportion = False):
    '''
    Exports json files of all timeline types, across all variables, dimensions, and recessions. Will overwrite any files already saved.

        Parameters: 
            basic (bool): determines if basic timelines will be generated. Default = False
            target (bool): determines if target timelines will be generated. High processing time. Default = False
            proportion (bool): determines if proportional timelines will be generated. Default = False
    
        Returns: 
            exported json files
    '''
    if basic:
        for dimension in ['area', 'industry']: 
            for variable in ['month3_emplvl', 'avg_wkly_wage', 'qtrly_estabs_count']: 
                for recession in [2001, 2008, 'full']: 
                    basic_timeline(variable = variable, dimension = dimension, recession = recession, save = True)
    if target:
        for dimension in ['area', 'industry']: 
            for variable in ['month3_emplvl', 'avg_wkly_wage', 'qtrly_estabs_count']: 
                for recession in [2001, 2008]: 
                    target_timeline(variable = variable, dimension = dimension, recession = recession, save = True, loadjson= basic )
    if proportion:
        for dimension in ['area', 'industry']: 
            for variable in ['month3_emplvl', 'avg_wkly_wage', 'qtrly_estabs_count']: 
                for recession in [2001, 2008]: 
                    proportional_timeline(variable = variable, dimension = dimension, recession = recession, save = True)                                                                                                                  


In [2]:
#abbreviations for variable and dimension names, used in filepaths
var_abbr = {'month3_emplvl': 'empl', 'avg_wkly_wage':'wage', 'qtrly_estabs_count':'firm'}
dim_abbr = {'industry':'indus', 'area':'area'}

#stores the relevant years for recessions. Will be expanded later.BOth str and int versions
recessions_str = {2001: ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007'],
2008: ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']}
recessions_int = {2001: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007],
2008: [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019],
'full': [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]}

#stores the quarter of the recession event
recession_events = {2001: 2001.75, 2008: 2008.75, 2020: 2020.25, 'full': None}

#reader-friendly event names
events_display = {2001: 'Sept 11', 2008: 'Financial Crash', 2020: 'COVID-19', 'full': None}

#column index for the final quarter in the recession timeline, crucial for charting
end_columns =  {2001:33, 2008:53, 'full':82}

#reader-friendly quarter designations for better charts- 2001
quarters_display = {2000.25: 'Q1 2000', 2000.5: 'Q2 2000', 2000.75: 'Q3 2000', 2001.0: 'Q4 2000', 2001.25: 'Q1 2001', 2001.5: 'Q2 2001', 2001.75: 'Q3 2001', 2002.0: 'Q4 2001', 2002.25: 'Q1 2002', 2002.5: 'Q2 2002', 2002.75: 'Q3 2002', 2003.0: 'Q4 2002', 2003.25: 'Q1 2003', 2003.5: 'Q2 2003', 2003.75: 'Q3 2003', 2004.0: 'Q4 2003', 2004.25: 'Q1 2004', 2004.5: 'Q2 2004', 2004.75: 'Q3 2004', 2005.0: 'Q4 2004', 2005.25: 'Q1 2005', 2005.5: 'Q2 2005', 2005.75: 'Q3 2005', 2006.0: 'Q4 2005', 2006.25: 'Q1 2006', 2006.5: 'Q2 2006', 2006.75: 'Q3 2006', 2007.0: 'Q4 2006', 2007.25: 'Q1 2007', 2007.5: 'Q2 2007', 2007.75: 'Q3 2007', 2008.0: 'Q4 2007', 2008.25: 'Q1 2008', 2008.5: 'Q2 2008', 2008.75: 'Q3 2008', 2009.0: 'Q4 2008', 2009.25: 'Q1 2009', 2009.5: 'Q2 2009', 2009.75: 'Q3 2009', 2010.0: 'Q4 2009', 2010.25: 'Q1 2010', 2010.5: 'Q2 2010', 2010.75: 'Q3 2010', 2011.0: 'Q4 2010', 2011.25: 'Q1 2011', 2011.5: 'Q2 2011', 2011.75: 'Q3 2011', 2012.0: 'Q4 2011', 2012.25: 'Q1 2012', 2012.5: 'Q2 2012', 2012.75: 'Q3 2012', 2013.0: 'Q4 2012', 2013.25: 'Q1 2013', 2013.5: 'Q2 2013', 2013.75: 'Q3 2013', 2014.0: 'Q4 2013', 2014.25: 'Q1 2014', 2014.5: 'Q2 2014', 2014.75: 'Q3 2014', 2015.0: 'Q4 2014', 2015.25: 'Q1 2015', 2015.5: 'Q2 2015', 2015.75: 'Q3 2015', 2016.0: 'Q4 2015', 2016.25: 'Q1 2016', 2016.5: 'Q2 2016', 2016.75: 'Q3 2016', 2017.0: 'Q4 2016', 2017.25: 'Q1 2017', 2017.5: 'Q2 2017', 2017.75: 'Q3 2017', 2018.0: 'Q4 2017', 2018.25: 'Q1 2018', 2018.5: 'Q2 2018', 2018.75: 'Q3 2018', 2019.0: 'Q4 2018', 2019.25: 'Q1 2019', 2019.5: 'Q2 2019', 2019.75: 'Q3 2019', 2020.0: 'Q4 2019', 2020.25: 'Q1 2020', 2020.5: 'Q2 2020', 2020.75: 'Q3 2020', 2021.0: 'Q4 2020'}

class Recession(object):
    """
    Stores relevant information for each recession.

    ...

    Attributes
    ----------
    years : list
        years included in recession analysis. Must be numeric
    event_year : int
        year in which recession event takes place
    event_quarter : float
        quarter in which recession event takes place
    event_label : str
        read-friendly description of recession event
    quarters : list
        list of quarters included in recession analysis
    xaxis : list
        list of reader-friendly quarters in recession analys
    import_schema :
        


    Methods
    -------
    derive_generation:
        Computes the industry's generation
    """

    def __init__(self, year = 2008):
        """
        Constructs necessary attributes for the recession

        Parameters
        ----------
            year : int (or 'full')
                year of recession, used as unique identifier
        """
        self.years = recessions_int[year]
        self.event_year = year
        self.event_quarter = recession_events[year]
        self.event_label = events_display[year]
        self.quarters = [quarter for quarter in quarters_display.keys() if quarter > min(self.years) and quarter <= (max(self.years) + 1)]
        self.xaxis = [v for k,v in quarters_display.items() if k in self.quarters]
        self.y_end = end_columns[year]


In [3]:
variable = 'qtrly_estabs_count' 
dimension = 'industry'
recession = 2008
save = False
loadjson = False

recession = Recession(recession)

if dimension == 'area':
    df = df[~df['area_fips'].str.contains("999")]
    index = ['area_fips', 'area_title']
elif dimension == 'industry':
    index = ['industry_code', 'industry_title']




In [4]:
df = basic_timeline(variable = variable, dimension = dimension, recession = recession.event_year, save = save)
df

qtrid,industry_code,industry_title,2007.25,2007.5,2007.75,2008.0,2008.25,2008.5,2008.75,2009.0,...,2017.75,2018.0,2018.25,2018.5,2018.75,2019.0,2019.25,2019.5,2019.75,2020.0
0,10,"Total, all industries",26815413.0,27035392.0,27270224.0,27450524.0,27314525.0,27407206.0,27553408.0,27622882.0,...,29852705.0,30067610.0,29928707.0,30152609.0,30360749.0,30652084.0,30558823.0,30810452.0,31068059.0,31332627.0
1,11,"Agriculture, forestry, fishing and hunting",95431.0,95578.0,96126.0,96430.0,95387.0,95331.0,95784.0,96049.0,...,105209.0,105517.0,105316.0,106084.0,106604.0,107012.0,106126.0,106874.0,107695.0,108412.0
2,21,"Mining, quarrying, and oil and gas extraction",28864.0,29247.0,29616.0,29975.0,30122.0,30544.0,30954.0,31219.0,...,32469.0,32518.0,32209.0,32358.0,32572.0,32799.0,32570.0,32553.0,32727.0,32679.0
3,22,Utilities,24755.0,24811.0,24842.0,24870.0,24847.0,24891.0,25018.0,24970.0,...,26697.0,26797.0,26856.0,27073.0,27195.0,27388.0,27356.0,27431.0,27560.0,27666.0
4,23,Construction,889775.0,897233.0,902561.0,899998.0,891237.0,891445.0,889112.0,881183.0,...,796124.0,800550.0,803317.0,813999.0,820521.0,827487.0,824947.0,833696.0,841448.0,846509.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,926150,Licensing and regulating commercial sectors,4557.0,4573.0,4596.0,4605.0,4603.0,4591.0,4649.0,4611.0,...,4433.0,4418.0,4337.0,4325.0,4314.0,4345.0,4340.0,4327.0,4324.0,4320.0
2473,927110,Space research and technology,128.0,129.0,131.0,135.0,137.0,137.0,138.0,140.0,...,104.0,105.0,101.0,97.0,97.0,95.0,95.0,97.0,97.0,96.0
2474,928110,National security,2920.0,2924.0,2910.0,2903.0,2900.0,2893.0,2892.0,2893.0,...,2657.0,2656.0,2630.0,2617.0,2614.0,2610.0,2599.0,2592.0,2593.0,2597.0
2475,928120,International affairs,260.0,260.0,261.0,267.0,264.0,262.0,263.0,264.0,...,345.0,353.0,351.0,348.0,351.0,354.0,354.0,348.0,349.0,350.0


In [5]:
df2 = df.drop(columns = index)
df2 = df2.reset_index()
df2

qtrid,index,2007.25,2007.5,2007.75,2008.0,2008.25,2008.5,2008.75,2009.0,2009.25,...,2017.75,2018.0,2018.25,2018.5,2018.75,2019.0,2019.25,2019.5,2019.75,2020.0
0,0,26815413.0,27035392.0,27270224.0,27450524.0,27314525.0,27407206.0,27553408.0,27622882.0,27130013.0,...,29852705.0,30067610.0,29928707.0,30152609.0,30360749.0,30652084.0,30558823.0,30810452.0,31068059.0,31332627.0
1,1,95431.0,95578.0,96126.0,96430.0,95387.0,95331.0,95784.0,96049.0,94824.0,...,105209.0,105517.0,105316.0,106084.0,106604.0,107012.0,106126.0,106874.0,107695.0,108412.0
2,2,28864.0,29247.0,29616.0,29975.0,30122.0,30544.0,30954.0,31219.0,31157.0,...,32469.0,32518.0,32209.0,32358.0,32572.0,32799.0,32570.0,32553.0,32727.0,32679.0
3,3,24755.0,24811.0,24842.0,24870.0,24847.0,24891.0,25018.0,24970.0,25070.0,...,26697.0,26797.0,26856.0,27073.0,27195.0,27388.0,27356.0,27431.0,27560.0,27666.0
4,4,889775.0,897233.0,902561.0,899998.0,891237.0,891445.0,889112.0,881183.0,848177.0,...,796124.0,800550.0,803317.0,813999.0,820521.0,827487.0,824947.0,833696.0,841448.0,846509.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,2472,4557.0,4573.0,4596.0,4605.0,4603.0,4591.0,4649.0,4611.0,4530.0,...,4433.0,4418.0,4337.0,4325.0,4314.0,4345.0,4340.0,4327.0,4324.0,4320.0
2473,2473,128.0,129.0,131.0,135.0,137.0,137.0,138.0,140.0,143.0,...,104.0,105.0,101.0,97.0,97.0,95.0,95.0,97.0,97.0,96.0
2474,2474,2920.0,2924.0,2910.0,2903.0,2900.0,2893.0,2892.0,2893.0,2887.0,...,2657.0,2656.0,2630.0,2617.0,2614.0,2610.0,2599.0,2592.0,2593.0,2597.0
2475,2475,260.0,260.0,261.0,267.0,264.0,262.0,263.0,264.0,248.0,...,345.0,353.0,351.0,348.0,351.0,354.0,354.0,348.0,349.0,350.0


In [6]:

#drops the index so that all columns are free of any type mismatches
df2 = df2.drop(columns = 'index')
df2 = df2.fillna(0)
df2

qtrid,2007.25,2007.50,2007.75,2008.00,2008.25,2008.50,2008.75,2009.00,2009.25,2009.50,...,2017.75,2018.00,2018.25,2018.50,2018.75,2019.00,2019.25,2019.50,2019.75,2020.00
0,26815413.0,27035392.0,27270224.0,27450524.0,27314525.0,27407206.0,27553408.0,27622882.0,27130013.0,27182490.0,...,29852705.0,30067610.0,29928707.0,30152609.0,30360749.0,30652084.0,30558823.0,30810452.0,31068059.0,31332627.0
1,95431.0,95578.0,96126.0,96430.0,95387.0,95331.0,95784.0,96049.0,94824.0,95065.0,...,105209.0,105517.0,105316.0,106084.0,106604.0,107012.0,106126.0,106874.0,107695.0,108412.0
2,28864.0,29247.0,29616.0,29975.0,30122.0,30544.0,30954.0,31219.0,31157.0,31133.0,...,32469.0,32518.0,32209.0,32358.0,32572.0,32799.0,32570.0,32553.0,32727.0,32679.0
3,24755.0,24811.0,24842.0,24870.0,24847.0,24891.0,25018.0,24970.0,25070.0,25119.0,...,26697.0,26797.0,26856.0,27073.0,27195.0,27388.0,27356.0,27431.0,27560.0,27666.0
4,889775.0,897233.0,902561.0,899998.0,891237.0,891445.0,889112.0,881183.0,848177.0,841087.0,...,796124.0,800550.0,803317.0,813999.0,820521.0,827487.0,824947.0,833696.0,841448.0,846509.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,4557.0,4573.0,4596.0,4605.0,4603.0,4591.0,4649.0,4611.0,4530.0,4474.0,...,4433.0,4418.0,4337.0,4325.0,4314.0,4345.0,4340.0,4327.0,4324.0,4320.0
2473,128.0,129.0,131.0,135.0,137.0,137.0,138.0,140.0,143.0,144.0,...,104.0,105.0,101.0,97.0,97.0,95.0,95.0,97.0,97.0,96.0
2474,2920.0,2924.0,2910.0,2903.0,2900.0,2893.0,2892.0,2893.0,2887.0,2876.0,...,2657.0,2656.0,2630.0,2617.0,2614.0,2610.0,2599.0,2592.0,2593.0,2597.0
2475,260.0,260.0,261.0,267.0,264.0,262.0,263.0,264.0,248.0,237.0,...,345.0,353.0,351.0,348.0,351.0,354.0,354.0,348.0,349.0,350.0


In [7]:
#specifies the lowest numbers during the recession. Disregards quarters before the recession event. 
df2['nadir'] = df2.iloc[:,6:].min(axis=1)

#specifies which quarter the nadir occured.
df2['nadir_time'] = (df2.iloc[:,6:].idxmin(axis=1).apply(lambda x: df.columns.get_loc(x)))-1

df2['nadir_qtr'] =  df2['nadir_time'] / 4 + recession.years[0]

#creates a column to store indices for lookup.
df2['new'] = [df2.iloc[i].values for i in df.index]

#specifies the highest numbers *before* the nadir.
df2['pre_peak'] = df2.apply(lambda x: max(x['new'][0:x['nadir_time']]), axis=1)

#specifies the highest numbers *after* the nadir
df2['post_peak'] = df2.apply(lambda x: max(x['new'][x['nadir_time']:]), axis=1) 

#specifies which quarter the pre-peak occurred.
df2['pre_peak_time'] = pd.Series([s[i] for i, s in zip(df2.index, df2['pre_peak'].apply(
    lambda x: [i for i in (df2.iloc[:,0:-6] == x)
            .idxmax(axis=1)]))]).apply(lambda x: df2.columns.get_loc(x)) + 1

df2['pre_peak_qtr'] = df2['pre_peak_time'] / 4 + recession.years[0]

#specifies which quarter the post-peak occurred.
df2['post_peak_time'] = pd.Series([s[i] for i, s in zip(df2.index, df2['post_peak'].apply(
    lambda x: [i for i in (df2.iloc[:,0:-6] == x)
            .idxmax(axis=1)]))]).apply(lambda x: df2.columns.get_loc(x)) + 1

df2['post_peak_qtr'] = df2['post_peak_time'] / 4 + recession.years[0]

#PRIMARY TARGET: did the area/industry achieve it's pre-recession peak?
df2['recovery'] = (df2['post_peak'] >= df2['pre_peak']) *1

df2

qtrid,2007.25,2007.5,2007.75,2008.0,2008.25,2008.5,2008.75,2009.0,2009.25,2009.5,...,nadir_time,nadir_qtr,new,pre_peak,post_peak,pre_peak_time,pre_peak_qtr,post_peak_time,post_peak_qtr,recovery
0,26815413.0,27035392.0,27270224.0,27450524.0,27314525.0,27407206.0,27553408.0,27622882.0,27130013.0,27182490.0,...,13,2010.25,"[26815413.0, 27035392.0, 27270224.0, 27450524....",27622882.0,31332627.00,8,2009.00,52,2020.00,1
1,95431.0,95578.0,96126.0,96430.0,95387.0,95331.0,95784.0,96049.0,94824.0,95065.0,...,13,2010.25,"[95431.0, 95578.0, 96126.0, 96430.0, 95387.0, ...",96430.0,108412.00,4,2008.00,52,2020.00,1
2,28864.0,29247.0,29616.0,29975.0,30122.0,30544.0,30954.0,31219.0,31157.0,31133.0,...,7,2008.75,"[28864.0, 29247.0, 29616.0, 29975.0, 30122.0, ...",30954.0,36355.00,7,2008.75,32,2015.00,1
3,24755.0,24811.0,24842.0,24870.0,24847.0,24891.0,25018.0,24970.0,25070.0,25119.0,...,8,2009.00,"[24755.0, 24811.0, 24842.0, 24870.0, 24847.0, ...",25018.0,27666.00,7,2008.75,52,2020.00,1
4,889775.0,897233.0,902561.0,899998.0,891237.0,891445.0,889112.0,881183.0,848177.0,841087.0,...,25,2013.25,"[889775.0, 897233.0, 902561.0, 899998.0, 89123...",902561.0,846509.00,3,2007.75,52,2020.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,4557.0,4573.0,4596.0,4605.0,4603.0,4591.0,4649.0,4611.0,4530.0,4474.0,...,47,2018.75,"[4557.0, 4573.0, 4596.0, 4605.0, 4603.0, 4591....",4789.0,4345.00,16,2011.00,48,2019.00,0
2473,128.0,129.0,131.0,135.0,137.0,137.0,138.0,140.0,143.0,144.0,...,27,2013.75,"[128.0, 129.0, 131.0, 135.0, 137.0, 137.0, 138...",150.0,2013.75,19,2011.75,1,2007.25,1
2474,2920.0,2924.0,2910.0,2903.0,2900.0,2893.0,2892.0,2893.0,2887.0,2876.0,...,28,2014.00,"[2920.0, 2924.0, 2910.0, 2903.0, 2900.0, 2893....",2924.0,2866.00,2,2007.50,33,2015.25,0
2475,260.0,260.0,261.0,267.0,264.0,262.0,263.0,264.0,248.0,237.0,...,10,2009.50,"[260.0, 260.0, 261.0, 267.0, 264.0, 262.0, 263...",267.0,2009.50,4,2008.00,1,2007.25,1


In [8]:
#create another dataframe to calculate the # of quarters to recover- only contains recovered datapoints
df3 = df2[df2['recovery'] == 1]

#creates a column to derive the recovery quarter- list of boolean values
df3['recovery_list'] = df3.apply(lambda x: (x['new'][x['nadir_time']:] >= x['pre_peak']), axis=1)

#creates a column for the number or quarters until the results pass the pre-peak high, since the nadir
df3['recovery_time'] = df3['recovery_list'].apply(lambda x: list(x).index(True)) + 1

df3['recovery_qtr'] = (df3['nadir_time'] + df3['recovery_time']) / 4 + recession.years[0]

df3

qtrid,2007.25,2007.5,2007.75,2008.0,2008.25,2008.5,2008.75,2009.0,2009.25,2009.5,...,pre_peak,post_peak,pre_peak_time,pre_peak_qtr,post_peak_time,post_peak_qtr,recovery,recovery_list,recovery_time,recovery_qtr
0,26815413.0,27035392.0,27270224.0,27450524.0,27314525.0,27407206.0,27553408.0,27622882.0,27130013.0,27182490.0,...,27622882.0,31332627.00,8,2009.00,52,2020.00,1,"[False, False, False, False, False, False, Tru...",7,2012.00
1,95431.0,95578.0,96126.0,96430.0,95387.0,95331.0,95784.0,96049.0,94824.0,95065.0,...,96430.0,108412.00,4,2008.00,52,2020.00,1,"[False, False, False, False, False, False, Tru...",7,2012.00
2,28864.0,29247.0,29616.0,29975.0,30122.0,30544.0,30954.0,31219.0,31157.0,31133.0,...,30954.0,36355.00,7,2008.75,32,2015.00,1,"[True, True, True, True, True, True, True, Tru...",1,2009.00
3,24755.0,24811.0,24842.0,24870.0,24847.0,24891.0,25018.0,24970.0,25070.0,25119.0,...,25018.0,27666.00,7,2008.75,52,2020.00,1,"[True, True, True, True, True, True, True, Tru...",1,2009.25
8,254021.0,255619.0,257088.0,257481.0,256539.0,256873.0,256766.0,256570.0,253522.0,252491.0,...,257481.0,298449.00,4,2008.00,52,2020.00,1,"[False, False, False, False, False, False, Fal...",13,2013.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,2441.0,2437.0,2432.0,2435.0,2466.0,2460.0,2496.0,2493.0,2509.0,2499.0,...,2534.0,2787.00,11,2009.75,49,2019.25,1,"[False, False, False, False, False, False, Fal...",13,2018.25
2467,1434.0,1432.0,1437.0,1425.0,1442.0,1449.0,1464.0,1472.0,1484.0,1480.0,...,1484.0,2013.00,9,2009.25,1,2007.25,1,"[False, False, False, False, False, False, Fal...",9,2015.25
2470,829.0,832.0,837.0,839.0,845.0,846.0,852.0,834.0,824.0,829.0,...,852.0,2013.00,7,2008.75,1,2007.25,1,"[False, False, False, False, False, True, True...",6,2014.50
2473,128.0,129.0,131.0,135.0,137.0,137.0,138.0,140.0,143.0,144.0,...,150.0,2013.75,19,2011.75,1,2007.25,1,"[False, False, False, False, False, False, Fal...",28,2020.75


In [9]:
#drops all redunant fields from the third dataframe
dropcols = merge_drop
if loadjson:
    for quarter in recession.quarters[1:]:
        dropcols.append(str(quarter))
else:
    dropcols.extend(recession.quarters[1:])
df3.drop(columns = dropcols, inplace=True)

#creates a new dataframe to store derived fields
df_new = df2[['nadir', 'nadir_qtr', 'nadir_time', 'pre_peak', 'pre_peak_time', 'pre_peak_qtr', 'post_peak', 'post_peak_time', 'post_peak_qtr', 'recovery']]

df_new

qtrid,nadir,nadir_qtr,nadir_time,pre_peak,pre_peak_time,pre_peak_qtr,post_peak,post_peak_time,post_peak_qtr,recovery
0,27008059.0,2010.25,13,27622882.0,8,2009.00,31332627.00,52,2020.00,1
1,94459.0,2010.25,13,96430.0,4,2008.00,108412.00,52,2020.00,1
2,30954.0,2008.75,7,30954.0,7,2008.75,36355.00,32,2015.00,1
3,24970.0,2009.00,8,25018.0,7,2008.75,27666.00,52,2020.00,1
4,744332.0,2013.25,25,902561.0,3,2007.75,846509.00,52,2020.00,0
...,...,...,...,...,...,...,...,...,...,...
2472,4314.0,2018.75,47,4789.0,16,2011.00,4345.00,48,2019.00,0
2473,85.0,2013.75,27,150.0,19,2011.75,2013.75,1,2007.25,1
2474,2550.0,2014.00,28,2924.0,2,2007.50,2866.00,33,2015.25,0
2475,237.0,2009.50,10,267.0,4,2008.00,2009.50,1,2007.25,1


In [10]:
#adds the recovery quarter column from the third dataframe
df_new = df_new.join(df3, how = 'left', rsuffix = '_recov')

#puts the computed points in a dataframe, joins with timeline
df = df.join(df_new, how = 'outer', rsuffix = '_derive')

#How many quarters did the jobs numbers decline?
df['decline_time'] = (df['nadir_time'] - df['pre_peak_time'])

df['growth_time'] = (df['post_peak_time'] - df['nadir_time'])

#Different in before/after jobs numbers
df['delta'] = df['post_peak'] - df['pre_peak']

In [11]:
df3

qtrid,nadir_time,pre_peak_time,post_peak_time,recovery_time,recovery_qtr
0,13,8,52,7,2012.00
1,13,4,52,7,2012.00
2,7,7,32,1,2009.00
3,8,7,52,1,2009.25
8,13,4,52,13,2013.50
...,...,...,...,...,...
2466,32,11,49,13,2018.25
2467,24,9,1,9,2015.25
2470,24,7,1,6,2014.50
2473,27,19,1,28,2020.75


In [12]:
#remove extra columns
df.drop(columns = ['nadir_time_recov', 'pre_peak_time_recov', 'post_peak_time_recov'], inplace = True)



In [14]:
#reorders columns for easier organization
col_order = index
col_order.extend(recession.quarters[1])
col_order.extend(['pre_peak', 'nadir', 'post_peak', 'recovery', 'delta', 'pre_peak_time', 'decline_time', 'nadir_time', 'post_peak_time', 'recovery_time', 'growth_time', 'pre_peak_qtr', 'nadir_qtr', 'post_peak_qtr', 'recovery_qtr', ])
col_order

['industry_code',
 'industry_title',
 2007.0,
 2007.25,
 2007.5,
 2007.75,
 2008.0,
 2008.25,
 2008.5,
 2008.75,
 2009.0,
 2009.25,
 2009.5,
 2009.75,
 2010.0,
 2010.25,
 2010.5,
 2010.75,
 2011.0,
 2011.25,
 2011.5,
 2011.75,
 2012.0,
 2012.25,
 2012.5,
 2012.75,
 2013.0,
 2013.25,
 2013.5,
 2013.75,
 2014.0,
 2014.25,
 2014.5,
 2014.75,
 2015.0,
 2015.25,
 2015.5,
 2015.75,
 2016.0,
 2016.25,
 2016.5,
 2016.75,
 2017.0,
 2017.25,
 2017.5,
 2017.75,
 2018.0,
 2018.25,
 2018.5,
 2018.75,
 2019.0,
 2019.25,
 2019.5,
 2019.75,
 2020.0,
 'pre_peak',
 'nadir',
 'post_peak',
 'recovery',
 'delta',
 'pre_peak_time',
 'decline_time',
 'nadir_time',
 'post_peak_time',
 'recovery_time',
 'growth_time',
 'pre_peak_qtr',
 'nadir_qtr',
 'post_peak_qtr',
 'recovery_qtr']

In [15]:
df

qtrid,industry_code,industry_title,2007.25,2007.5,2007.75,2008.0,2008.25,2008.5,2008.75,2009.0,...,pre_peak_qtr,post_peak,post_peak_time,post_peak_qtr,recovery,recovery_time,recovery_qtr,decline_time,growth_time,delta
0,10,"Total, all industries",26815413.0,27035392.0,27270224.0,27450524.0,27314525.0,27407206.0,27553408.0,27622882.0,...,2009.00,31332627.00,52,2020.00,1,7.0,2012.00,5,39,3709745.00
1,11,"Agriculture, forestry, fishing and hunting",95431.0,95578.0,96126.0,96430.0,95387.0,95331.0,95784.0,96049.0,...,2008.00,108412.00,52,2020.00,1,7.0,2012.00,9,39,11982.00
2,21,"Mining, quarrying, and oil and gas extraction",28864.0,29247.0,29616.0,29975.0,30122.0,30544.0,30954.0,31219.0,...,2008.75,36355.00,32,2015.00,1,1.0,2009.00,0,25,5401.00
3,22,Utilities,24755.0,24811.0,24842.0,24870.0,24847.0,24891.0,25018.0,24970.0,...,2008.75,27666.00,52,2020.00,1,1.0,2009.25,1,44,2648.00
4,23,Construction,889775.0,897233.0,902561.0,899998.0,891237.0,891445.0,889112.0,881183.0,...,2007.75,846509.00,52,2020.00,0,NaN,NaN,22,27,-56052.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,926150,Licensing and regulating commercial sectors,4557.0,4573.0,4596.0,4605.0,4603.0,4591.0,4649.0,4611.0,...,2011.00,4345.00,48,2019.00,0,NaN,NaN,31,1,-444.00
2473,927110,Space research and technology,128.0,129.0,131.0,135.0,137.0,137.0,138.0,140.0,...,2011.75,2013.75,1,2007.25,1,28.0,2020.75,8,-26,1863.75
2474,928110,National security,2920.0,2924.0,2910.0,2903.0,2900.0,2893.0,2892.0,2893.0,...,2007.50,2866.00,33,2015.25,0,NaN,NaN,26,5,-58.00
2475,928120,International affairs,260.0,260.0,261.0,267.0,264.0,262.0,263.0,264.0,...,2008.00,2009.50,1,2007.25,1,7.0,2011.25,6,-9,1742.50


In [16]:
df.columns

Index([ 'industry_code', 'industry_title',          2007.25,           2007.5,
                2007.75,           2008.0,          2008.25,           2008.5,
                2008.75,           2009.0,          2009.25,           2009.5,
                2009.75,           2010.0,          2010.25,           2010.5,
                2010.75,           2011.0,          2011.25,           2011.5,
                2011.75,           2012.0,          2012.25,           2012.5,
                2012.75,           2013.0,          2013.25,           2013.5,
                2013.75,           2014.0,          2014.25,           2014.5,
                2014.75,           2015.0,          2015.25,           2015.5,
                2015.75,           2016.0,          2016.25,           2016.5,
                2016.75,           2017.0,          2017.25,           2017.5,
                2017.75,           2018.0,          2018.25,           2018.5,
                2018.75,           2019.0,          

In [17]:
df.reindex(columns = col_order)


qtrid,industry_code,industry_title,2007.0,2007.25,2007.5,2007.75,2008.0,2008.25,2008.5,2008.75,...,pre_peak_time,decline_time,nadir_time,post_peak_time,recovery_time,growth_time,pre_peak_qtr,nadir_qtr,post_peak_qtr,recovery_qtr
0,10,"Total, all industries",NaN,26815413.0,27035392.0,27270224.0,27450524.0,27314525.0,27407206.0,27553408.0,...,8,5,13,52,7.0,39,2009.00,2010.25,2020.00,2012.00
1,11,"Agriculture, forestry, fishing and hunting",NaN,95431.0,95578.0,96126.0,96430.0,95387.0,95331.0,95784.0,...,4,9,13,52,7.0,39,2008.00,2010.25,2020.00,2012.00
2,21,"Mining, quarrying, and oil and gas extraction",NaN,28864.0,29247.0,29616.0,29975.0,30122.0,30544.0,30954.0,...,7,0,7,32,1.0,25,2008.75,2008.75,2015.00,2009.00
3,22,Utilities,NaN,24755.0,24811.0,24842.0,24870.0,24847.0,24891.0,25018.0,...,7,1,8,52,1.0,44,2008.75,2009.00,2020.00,2009.25
4,23,Construction,NaN,889775.0,897233.0,902561.0,899998.0,891237.0,891445.0,889112.0,...,3,22,25,52,NaN,27,2007.75,2013.25,2020.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,926150,Licensing and regulating commercial sectors,NaN,4557.0,4573.0,4596.0,4605.0,4603.0,4591.0,4649.0,...,16,31,47,48,NaN,1,2011.00,2018.75,2019.00,NaN
2473,927110,Space research and technology,NaN,128.0,129.0,131.0,135.0,137.0,137.0,138.0,...,19,8,27,1,28.0,-26,2011.75,2013.75,2007.25,2020.75
2474,928110,National security,NaN,2920.0,2924.0,2910.0,2903.0,2900.0,2893.0,2892.0,...,2,26,28,33,NaN,5,2007.50,2014.00,2015.25,NaN
2475,928120,International affairs,NaN,260.0,260.0,261.0,267.0,264.0,262.0,263.0,...,4,6,10,1,7.0,-9,2008.00,2009.50,2007.25,2011.25


In [ ]:
df.columns